In [ ]:
## This code was last modified on 27/11/2025

#The code generates the energy spectrum of the 0+1 supersymmetric quantum mechanics Hamiltonian for three supersymmetric potentials (HO, DW, and AHO).
#The energy levels are plotted as a function of the parameter g, which appears only in the DW and AHO case.

In [ ]:
# Essential information about the structure of the code for each block "approximately"
"""
Code structure
0) User choices block
1) Load all libraries modules
2) Calculate and digonalize the Hamiltonian for different truncation level (\Lambda)
3) Plot the energy spectrum vs the g parameter
"""

In [ ]:
## Define user choice

# Parameters for the construction of the Hamiltonian
superpotential = "AHO"                   # Chose a superpontentials "HO" Harmonic Oscillator; "DW" Double Well; "AHO" Anharmonic Oscillator 
N_bosonic_modes = 512                     # Chose the number of bosonic modes allowed (\Lambda)
m = 2.0                                   # Chose the bosonic and fermionic mass
mu = 3.0                               # Chose the mu interaction strenght present in DW

# Enter the min and max g value needed for plotting the energies versus g
g_min = 0
g_max = 20.0
g_steps = 100


H_full_or_block = "Full"                # Chose the "Full" for the full Hamiltonian or "Block" for the lower block

n_energy_states = 6                     # Chose the number of energy level to calculate                

In [ ]:
## Section importing all needed libraries modules

#Packages for interacting with the operating system
import os

import sys
#Append the code's directory to the interpreter to search for files
sys.path.append('..')

#Importing the Hamiltonian from the file (Hamiltonian_SQM_0p1.py) inside the folder (hamiltonian_SQM)
from hamiltonian_SQM.Hamiltonian_SQM_0p1 import *

#Packages for numerical calculation
import numpy as np

from scipy.sparse.linalg import eigs

#For eigenvalues and eigenvectors
from numpy import linalg 


#Packages for plotting
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, FixedFormatter

#Needed to get the time and date
import time
from datetime import datetime


## Not fully necessary modules
# Load basic sound playing machine in Windows
import winsound

In [ ]:
#Defining the Hamiltonian name for plots and files
Hamiltonian_name = superpotential+"_"+H_full_or_block


# list of values for g srenght
g_values=np.linspace(g_min, g_max, g_steps)
#print(g_values)


selected_eigenvalues=[]

for g in g_values:
    
    # Calculate the Hamiltonian 
    hamiltonian = Hamiltonian_SQM_0p1(superpotential, n_bosonic_modes = N_bosonic_modes, m = m, g = g, mu = mu)

    #Dictionary to handle the selection of full or block Hamiltonian and generation of the dense array
    H_full_or_block_dict = {
        "Full": hamiltonian.toarray(),
        "Block": hamiltonian.toarray()[N_bosonic_modes:2*N_bosonic_modes,N_bosonic_modes:2*N_bosonic_modes]
        }

    hamiltonian_array = H_full_or_block_dict.get(H_full_or_block, "Choice not present! type " + str([i for i in H_full_or_block_dict.keys()]))
    

    # Calculate the eigenvalue and eigenvetor
    eigenvalues, eigenVector = np.linalg.eig(hamiltonian_array)
    
    #Sort the eingenvalues 
    sorted_eigenvalues=np.sort(eigenvalues).real
    #print(sorted_eigenvalues)
    
    selected_eigenvalues.append(sorted_eigenvalues[:n_energy_states])


eigenvalue_array = np.vstack(selected_eigenvalues)
#print(eigenvalue_array)

#===================================================================================================================================================================


In [ ]:
#===================================================================================================================================================================

#Plot section

# Enable LaTeX-style formatting globally
plt.rcParams['text.usetex'] = True

#Selection of standard colors used in the following plots
colors_list = ["forestgreen", "skyblue", "indigo", 'orange', "violet", "cyan", "darkgray", "dodgerblue","sienna", "lawngreen", "magenta", "teal", "gold" ]


em_spectrum_colors_6 = [
    '#FF0000',  # Red
    '#FFA500',  # Orange
    '#32CD32',  # Green
    '#1E90FF',  # Blue
    '#4B0082',  # Indigo
    '#8B00FF',  # Violet

]



#Selection of standard marker symbols
markers_list = ['o',  # Circle
           's',  # Square
           '^',  # Triangle Up
           'D',  # Diamond
           'h', # Hexagon
           '*',  # Star
            'v',  # Triangle Down
           'x',  # X
           '+',  # Plus
           'p'  # Pentagon
           ]  



plt.figure(figsize=(18,12))

"""
#Choose log scale for the y-axis if the ground-state energy is larger than zero otherwise symlog (logarithmic scaling with negative values)
if all( i > 0 for i in eigenvalue_array[:,0]):
    plt.yscale('log')
else:
    plt.yscale('symlog')
"""

# Generate state labels 
state_labels= [rf"${i+1} \rm st$" if i == 0 else rf"""${i+1} \rm nd$""" if i == 1 else rf"""${i+1} \rm rd$""" if i == 2 else rf"""${i+1} \rm th$""" for i in range(n_energy_states)]


# Plot the eigenvalues versus the g parameter
for i in range(n_energy_states):
    plt.plot(g_values, eigenvalue_array[:,i], color=em_spectrum_colors_6[i], lw=1.0, marker = markers_list[i], markersize =8,  label=state_labels[i])


#Plot title and axis labels
fontsize = 50

#plt.title(f"Smallest VQD eigenspectrum convergence history (No Shots) \n" + plot_info_title, fontsize = fontsize)

# Define a standard title info for all plots
plot_title_dict = {
        "HO": str(f"{Hamiltonian_name.replace('_',' ')}      "+r"( $\frac{1}{2}m q^2$ )      " + r"$\Lambda$"+f" = {N_bosonic_modes}      "+ r"$m$"+f" = {m}"),
        "DW": str(f"{Hamiltonian_name.replace('_',' ')}      "+r"(  $\frac{1}{2}m q^2 +g \left( \frac{1}{3} q^3 + \mu^2 q \right)$ )      "  + r"$\Lambda$"+f" = {N_bosonic_modes}      "+ r"$m$"+f" = {m}      " + r"$\mu$"+f" = {mu}") ,
        "AHO": str(f"{Hamiltonian_name.replace('_',' ')}      "+r"( $\frac{1}{2}m q^2 + \frac{1}{4} g q^4 $ )      "  + r"$\Lambda$"+f" = {N_bosonic_modes}      "+ r"$m$"+f" = {m}")
        }


plot_info_title = plot_title_dict.get(superpotential, "Choice not present! type " + str([i for i in plot_title_dict.keys()]))


#plt.title(plot_info_title, fontsize = fontsize)

plt.xlabel(r"$ \rm g$", fontsize = 50, labelpad=10)
plt.ylabel(r"""$ \rm energy$""",fontsize = 50, labelpad=10)
plt.xticks(fontsize = fontsize) 
plt.yticks(fontsize = fontsize) 

#plt.legend(fontsize = 35, loc='center right')


plt.legend(fontsize = 35, loc='center right',      # Anchor point of the legend
    bbox_to_anchor=(1.0, 0.4),     # (x, y) in axes coordinates (0 = left, 1 = top)
)


#Defijnhe a name for the plot file
file_name_dict = {
        "HO": "E_spectrum_"+Hamiltonian_name+ f"__nbm_{N_bosonic_modes}__m_{m}" + "__g_"+f"{g_min, g_max, g_steps}" +"_.pdf",
        "DW": "E_spectrum_"+Hamiltonian_name+ f"__nbm_{N_bosonic_modes}__m_{m}__mu_{mu}" + "__g_"+f"{g_min, g_max, g_steps}" +"_.pdf",
        "AHO":"E_spectrum_"+Hamiltonian_name+ f"__nbm_{N_bosonic_modes}__m_{m}" + "__g_"+f"{g_min, g_max, g_steps}" +"_.pdf"
        }


#==========================================
# x ticks
#plt.xticks([0.0, 0.1, 0.2, 0.3, 0.4])

#plt.xlim(0, 0.25)
#==========================================

#plt.ylim([0.1, 1e1])
plt.yticks(fontsize = 45) 
plt.xticks(fontsize = 45) 

ax = plt.gca()  # Get current Axes instance
ax.set_yscale('symlog', linthresh=0.5)  # If you're plotting in log scale
ax.set_ylim([-0.1, 1e2])
ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[1,2,3,4,5, 6,7,8,9], numticks=10))
#ax.tick_params(which='both', direction='in', right=True)


plt.savefig(file_name_dict.get(superpotential, "Choice not present! type " + str([i for i in plot_title_dict.keys()])), bbox_inches='tight', format='pdf', dpi=600)

plt.show()

plt.close() 
#=======================================================================================================================
